In [19]:
import ifcopenshell
from topologicpy.Topology import Topology
from topologicpy.Dictionary import Dictionary

# Beispielhafte Nutzung
#ifc_file_path = '../Hus28_test.ifc'
ifc_file_path = '../Test_Walls.ifc'
storey_name = 'Plan 10'

# IFC-Datei öffnen
ifc_file = ifcopenshell.open(ifc_file_path)
print("IFC file opened successfully.")

# Liste, um die GUIDs der Wände im angegebenen Geschoss zu speichern
guids_in_storey = []

# Durchlaufen aller IfcWall-Objekte und deren Beziehungen
for wall in ifc_file.by_type('IfcWall'):
    for rel in ifc_file.by_type("IfcRelContainedInSpatialStructure"):
        if wall in rel.RelatedElements:
            storey = rel.RelatingStructure
            if storey and storey.Name == storey_name:
                guids_in_storey.append(wall.GlobalId)
                break

print(f"Found {len(guids_in_storey)} GUIDs in storey '{storey_name}'.")

# Laden der IFC-Datei
ifc_file = ifcopenshell.open(ifc_file_path)
print("IFC file opened successfully.")

# Erstellen der Topologien aller Wände im IFC-Modell
wall_topologies = Topology.ByIFCFile(file=ifc_file, transferDictionaries=True, includeTypes=['IfcWall'])
print("Initial wall topologies created successfully.")

# Filtern der Topologien nach GUIDs der Wände im Geschoss
walls_in_storey = []

for topo in wall_topologies:
    topo_dict = Topology.Dictionary(topo)
    topo_guid = Dictionary.ValueAtKey(topo_dict, "IFC_guid")
    if topo_guid in guids_in_storey:
        walls_in_storey.append(topo)

print(f"Found {len(walls_in_storey)} wall topologies in storey '{storey_name}'.")



IFC file opened successfully.
Found 7 GUIDs in storey 'Plan 10'.
IFC file opened successfully.
Initial wall topologies created successfully.
Found 7 wall topologies in storey 'Plan 10'.


In [20]:
import re

def filter_ifcspaces_by_storey(spaces, storey_name):
    filtered_spaces = []
    for space in spaces:
        for rel in space.Decomposes:
            if rel.is_a("IfcRelAggregates") and rel.RelatingObject.is_a("IfcBuildingStorey"):
                if rel.RelatingObject.Name == storey_name:
                    filtered_spaces.append(space)
                    break
    return filtered_spaces

def filter_spaces_by_name(spaces):
    digit_pattern = re.compile(r'^\d{1,5}$')
    filtered_spaces = []
    for space in spaces:
        if digit_pattern.match(space.Name):
            filtered_spaces.append(space)
    return filtered_spaces

# Alle IfcSpaces einlesen
spaces = ifc_file.by_type('IfcSpace')

# Nach Stockwerk filtern
spaces_in_storey = filter_ifcspaces_by_storey(spaces, storey_name)
print(f"Filtered {len(spaces_in_storey)} IfcSpaces in storey '{storey_name}'.")

# Nach Namen filtern
filtered_spaces = filter_spaces_by_name(spaces_in_storey)
print(f"Filtered {len(filtered_spaces)} IfcSpaces with valid names.")

Filtered 2 IfcSpaces in storey 'Plan 10'.
Filtered 2 IfcSpaces with valid names.


In [21]:
# Erstellen der Topologien aller gefilterten Räume im IFC-Modell
space_topologies = Topology.ByIFCFile(file=ifc_file, transferDictionaries=True, includeTypes=['IfcSpace'])
print("Initial space topologies created successfully.")

# Filtern der Topologien nach den gefilterten Räumen
spaces_in_storey = []

for topo in space_topologies:
    topo_dict = Topology.Dictionary(topo)
    topo_guid = Dictionary.ValueAtKey(topo_dict, "IFC_guid")
    if any(space.GlobalId == topo_guid for space in filtered_spaces):
        spaces_in_storey.append(topo)

print(f"Found {len(spaces_in_storey)} space topologies in storey '{storey_name}' with valid names.")

Initial space topologies created successfully.
Found 2 space topologies in storey 'Plan 10' with valid names.


In [22]:
# Wall Topologies
print(len(walls_in_storey))

# Wall Topologies
print(len(spaces_in_storey))

# Wall Guids
print(len(guids_in_storey))



7
2
7


In [23]:
from neo4j import GraphDatabase

# Verbindung zur Neo4J-Datenbank herstellen
uri = "bolt://localhost:7687"  # Standardadresse für Neo4J
username = "neo4j"
password = "testpass"
driver = GraphDatabase.driver(uri, auth=(username, password))

def create_wall_nodes(driver, topologies):
    with driver.session() as session:
        for topo in topologies:
            topo_dict = Topology.Dictionary(topo)
            topo_guid = Dictionary.ValueAtKey(topo_dict, "IFC_guid")
            wall_name = Dictionary.ValueAtKey(topo_dict, "IFC_Name")
            wall_type = Dictionary.ValueAtKey(topo_dict, "IFC_Type")
            
            # Erstellen des Cypher-Statements
            cypher_query = """
            MERGE (w:Wall {guid: $guid})
            SET w.name = $name, w.type = $type
            """
            session.run(cypher_query, guid=topo_guid, name=wall_name, type=wall_type)
            print(f"Wall node with GUID {topo_guid} created or updated.")

# Wall Nodes in der Datenbank anlegen
create_wall_nodes(driver, walls_in_storey)


Wall node with GUID 3iln0jDazAuRap$f6vl5mw created or updated.
Wall node with GUID 3iln0jDazAuRap$f6vl5mu created or updated.
Wall node with GUID 3iln0jDazAuRap$f6vl5ms created or updated.
Wall node with GUID 3iln0jDazAuRap$f6vl5mq created or updated.
Wall node with GUID 3iln0jDazAuRap$f6vl5sP created or updated.
Wall node with GUID 3iln0jDazAuRap$f6vl5mi created or updated.
Wall node with GUID 3iln0jDazAuRap$f6vl5mo created or updated.


In [24]:
# Funktion zum Erstellen von Raum-Nodes in Neo4J
def create_room_nodes(driver, space_topologies):
    with driver.session() as session:
        for topo in space_topologies:
            topo_dict = Topology.Dictionary(topo)
            room_guid = Dictionary.ValueAtKey(topo_dict, "IFC_guid")
            room_name = Dictionary.ValueAtKey(topo_dict, "IFC_Name")
            
            # Erstellen des Raum-Nodes
            cypher_query = """
            MERGE (r:Room {guid: $room_guid})
            SET r.name = $room_name
            """
            session.run(cypher_query, room_guid=room_guid, room_name=room_name)
            print(f"Room node with GUID {room_guid} created or updated.")

# Räume in der Neo4J-Datenbank anlegen
create_room_nodes(driver, spaces_in_storey)

Room node with GUID 0mWnWA_Rb6gRnszfQSZLg9 created or updated.
Room node with GUID 0mWnWA_Rb6gRnszfQSZLgB created or updated.


In [25]:
# Funktion zum Erstellen von Material Nodes und Beziehungen in Neo4J
def create_material_nodes_and_relationships(driver, wall_topologies):
    wall_materials_dict = {}  # Dictionary zum Speichern der Materialien pro Wand

    with driver.session() as session:
        for topo in wall_topologies:
            topo_dict = Topology.Dictionary(topo)
            wall_guid = Dictionary.ValueAtKey(topo_dict, "IFC_guid")

            # Open IFC file again to extract material layers
            wall = ifc_file.by_guid(wall_guid)
            if not wall:
                print(f"Wall with GUID {wall_guid} not found.")
                continue

            material_names = []

            # Extract materials associated with the wall
            for rel in wall.HasAssociations:
                if rel.is_a('IfcRelAssociatesMaterial'):
                    material_constituent_set = rel.RelatingMaterial
                    if material_constituent_set.is_a('IfcMaterialConstituentSet'):
                        for constituent in material_constituent_set.MaterialConstituents:
                            material_name = constituent.Material.Name if constituent.Material else "No material name"
                            material_names.append(material_name)

            # Füge die Materialien dem Dictionary hinzu
            wall_materials_dict[wall_guid] = material_names

            if material_names:
                print(f"Materials for wall {wall_guid}: {material_names}")

                # Create Material nodes and relationships for each material layer
                for i, material_name in enumerate(material_names):
                    # Erstelle einen eindeutigen Materialnamen pro Wand
                    unique_material_name = f"{wall_guid}_{material_name}_{i}"

                    # Create unique Material node
                    cypher_query = """
                    MERGE (m:Material {unique_name: $unique_material_name})
                    SET m.Material = $material_name
                    """
                    session.run(cypher_query, unique_material_name=unique_material_name, material_name=material_name)

                    # Create relationship between Wall and Material
                    cypher_query = """
                    MATCH (w:Wall {guid: $wall_guid}), (m:Material {unique_name: $unique_material_name})
                    MERGE (w)-[:ConsistsOf]->(m)
                    """
                    session.run(cypher_query, wall_guid=wall_guid, unique_material_name=unique_material_name)

                    # Create a one-way InternallyConnected relationship between adjacent materials
                    if i > 0:
                        previous_material_name = f"{wall_guid}_{material_names[i-1]}_{i-1}"
                        cypher_query = """
                        MATCH (m1:Material {unique_name: $previous_material_name}), (m2:Material {unique_name: $current_material_name})
                        MERGE (m1)-[:InternallyConnected]->(m2)
                        """
                        session.run(cypher_query, previous_material_name=previous_material_name, current_material_name=unique_material_name)
            else:
                print(f"No materials found for wall {wall_guid}.")

    return wall_materials_dict  # Rückgabe des Dictionaries

# Materialien und Beziehungen zur Neo4J-Datenbank hinzufügen
wall_materials_dict = create_material_nodes_and_relationships(driver, walls_in_storey)

# Überprüfung des erstellten Dictionaries
print("Wall materials dictionary:")
for wall_guid, materials in wall_materials_dict.items():
    print(f"Wall {wall_guid}: {materials}")

Materials for wall 3iln0jDazAuRap$f6vl5mw: ['Gypsum Wall Board', 'Gypsum Wall Board', 'Metal Stud Layer']
Materials for wall 3iln0jDazAuRap$f6vl5mu: ['Concrete Masonry Units', 'Air', 'Rigid insulation', 'Concrete Masonry Units', 'Metal Furring', 'Gypsum Wall Board']
Materials for wall 3iln0jDazAuRap$f6vl5ms: ['Siding, Clapboard', 'Plywood, Sheathing', 'Softwood, Lumber', 'Gypsum Wall Board']
Materials for wall 3iln0jDazAuRap$f6vl5mq: ['Metal Stud Layer', 'Gypsum Wall Board']
Materials for wall 3iln0jDazAuRap$f6vl5sP: ['Concrete, Lightweight', 'Concrete, Cast-in-Place gray', 'Concrete, Precast']
Materials for wall 3iln0jDazAuRap$f6vl5mi: ['Clad - White', 'Concrete Masonry Units']
Materials for wall 3iln0jDazAuRap$f6vl5mo: ['Gypsum Wall Board', 'Gypsum Wall Board', 'Softwood, Lumber']
Wall materials dictionary:
Wall 3iln0jDazAuRap$f6vl5mw: ['Gypsum Wall Board', 'Gypsum Wall Board', 'Metal Stud Layer']
Wall 3iln0jDazAuRap$f6vl5mu: ['Concrete Masonry Units', 'Air', 'Rigid insulation', 'Con

In [44]:
from topologicpy.Topology import Topology
from topologicpy.Vertex import Vertex
from topologicpy.Dictionary import Dictionary

def calculate_dimensions(vertices):
    # Extract the coordinates of the vertices
    x_coords = [Vertex.X(v) for v in vertices]
    y_coords = [Vertex.Y(v) for v in vertices]
    z_coords = [Vertex.Z(v) for v in vertices]

    # Calculate the dimensions based on the minimum and maximum coordinate values
    length = max(x_coords) - min(x_coords)
    thickness = max(y_coords) - min(y_coords)
    height = max(z_coords) - min(z_coords)

    return length, thickness, height

for wall in walls_in_storey:

    wall_dict = Dictionary.ValueAtKey(Topology.Dictionary(wall), key="IFC_guid")

    print(f"Processing Wall {wall_dict}...")  # Print the current wall number
    
    cells_in_wall = Topology.Cells(wall)  # Decompose 'wall' into its cells
    
    for cell in cells_in_wall:
        vertices = Topology.Vertices(cell)  # Get the vertices of the cell
        
        if len(vertices) >= 4:  # At least 4 vertices are required for a rectangle
            length, thickness, height = calculate_dimensions(vertices)
            print(f"Cell dimensions - Length: {length:.4f}, Thickness: {thickness:.4f}, Height: {height:.2f}")
        else:
            print("The cell does not have enough vertices to calculate length, thickness, and height.")


Processing Wall 3iln0jDazAuRap$f6vl5mw...
Cell dimensions - Length: 5.0305, Thickness: 0.0600, Height: 6.10
Cell dimensions - Length: 5.6355, Thickness: 0.0060, Height: 6.10
Cell dimensions - Length: 5.6355, Thickness: 0.0125, Height: 6.10
Processing Wall 3iln0jDazAuRap$f6vl5mu...
Cell dimensions - Length: 0.0160, Thickness: 4.3355, Height: 6.10
Cell dimensions - Length: 0.0500, Thickness: 4.3355, Height: 6.10
Cell dimensions - Length: 0.2900, Thickness: 4.3955, Height: 6.10
Cell dimensions - Length: 0.0750, Thickness: 4.6455, Height: 6.10
Cell dimensions - Length: 0.0500, Thickness: 4.6455, Height: 6.10
Cell dimensions - Length: 0.1900, Thickness: 4.6455, Height: 6.10
Processing Wall 3iln0jDazAuRap$f6vl5ms...
Cell dimensions - Length: 0.0125, Thickness: 4.3770, Height: 6.10
Cell dimensions - Length: 0.0125, Thickness: 4.3770, Height: 6.10
Cell dimensions - Length: 0.1400, Thickness: 4.6160, Height: 6.10
Cell dimensions - Length: 0.0125, Thickness: 4.6160, Height: 6.10
Cell dimensions 

In [26]:
import ifcopenshell
from topologicpy.Topology import Topology
from topologicpy.Dictionary import Dictionary
from neo4j import GraphDatabase

# Funktion zur Überprüfung von Berührungen zwischen Zelle und Räumen
def check_touching_spaces(cell, space_topologies):
    for space_topo in space_topologies:
        merged_topo = Topology.Merge(cell, space_topo)
        shared_faces = Topology.SharedFaces((Topology.Cells(merged_topo)[0]), (Topology.Cells(merged_topo)[1]))
        if shared_faces:
            return Dictionary.ValueAtKey(Topology.Dictionary(space_topo), "IFC_guid")
    return None

# Funktion zur Erstellung von "IsFacing"-Beziehungen in Neo4J
def add_is_facing_relationships(driver, wall_topologies, space_topologies, wall_materials_dict):
    with driver.session() as session:
        for wall_topo in wall_topologies:
            topo_dict = Topology.Dictionary(wall_topo)
            wall_guid = Dictionary.ValueAtKey(topo_dict, "IFC_guid")

            # Materialliste für die aktuelle Wand abrufen
            materials = wall_materials_dict.get(wall_guid, [])
            if not materials:
                print(f"No materials found for wall {wall_guid}.")
                continue

            # Zellen der Wandtopologie abrufen und invertieren
            wall_cells = Topology.Cells(wall_topo)[::-1]

            if len(wall_cells) < 2:
                print(f"Wall with GUID {wall_guid} does not have enough cells for processing.")
                continue

            # Erste (äußerste) und letzte (innerste) Zelle in der invertierten Liste
            outer_cell = wall_cells[0]
            inner_cell = wall_cells[-1]

            # Berührungen für äußere und innere Schicht prüfen und Beziehungen erstellen
            def create_is_facing_relationship(cell, material_name, layer_index):
                space_guid = check_touching_spaces(cell, space_topologies)
                if space_guid:
                    unique_material_name = f"{wall_guid}_{material_name}_{layer_index}"
                    cypher_query = """
                    MATCH (m:Material {unique_name: $material_name}), (r:Room {guid: $space_guid})
                    MERGE (m)-[:IsFacing]->(r)
                    """
                    session.run(cypher_query, material_name=unique_material_name, space_guid=space_guid)
                    print(f"Added IsFacing relationship between Material {unique_material_name} and Room {space_guid}")

            # Erstellen der Beziehung für die äußere Schicht (Index 0)
            create_is_facing_relationship(outer_cell, materials[0], 0)

            # Erstellen der Beziehung für die innere Schicht (Index n-1)
            create_is_facing_relationship(inner_cell, materials[-1], len(materials) - 1)

# Nutzung der Funktion zum Hinzufügen von IsFacing Beziehungen
add_is_facing_relationships(driver, walls_in_storey, spaces_in_storey, wall_materials_dict)


Added IsFacing relationship between Material 3iln0jDazAuRap$f6vl5mw_Metal Stud Layer_2 and Room 0mWnWA_Rb6gRnszfQSZLg9
Added IsFacing relationship between Material 3iln0jDazAuRap$f6vl5mu_Gypsum Wall Board_5 and Room 0mWnWA_Rb6gRnszfQSZLg9
Added IsFacing relationship between Material 3iln0jDazAuRap$f6vl5ms_Gypsum Wall Board_3 and Room 0mWnWA_Rb6gRnszfQSZLgB
Added IsFacing relationship between Material 3iln0jDazAuRap$f6vl5mq_Metal Stud Layer_0 and Room 0mWnWA_Rb6gRnszfQSZLgB
Added IsFacing relationship between Material 3iln0jDazAuRap$f6vl5sP_Concrete, Lightweight_0 and Room 0mWnWA_Rb6gRnszfQSZLg9
Added IsFacing relationship between Material 3iln0jDazAuRap$f6vl5mi_Concrete Masonry Units_1 and Room 0mWnWA_Rb6gRnszfQSZLg9
Added IsFacing relationship between Material 3iln0jDazAuRap$f6vl5mo_Softwood, Lumber_2 and Room 0mWnWA_Rb6gRnszfQSZLgB


In [27]:
# Treiber schließen
driver.close()